In [119]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import wikipediaapi as wiki
from datetime import datetime
import random
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import wikipedia
import time
import urllib
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException

import time

In [ ]:
##########################
###     CONSTANTS      ###
##########################

In [160]:
# IMdb API key
api_key = "k_11m5rq35"

# headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive',
        "referer":"referer: https://www.google.com/",
        "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
        }

cookies = {"CONSENT":"YES+shp.gws-20210330-0-RC1.de+FX+412"}

# options = Options()
# options.headless = True

# Parent and Relative strings to search for in wikipedia infobox
parent_pattern = re.compile('Parent')
relative_pattern = re.compile('Relative')
starring_pattern = re.compile('Starring')
voice_pattern = re.compile('Voices of')
wikipedia_pattern = re.compile("wiki")

not_nepo_relationships = 'son|daughter|nephew|niece|grandson|grand-daughter|grandnephew|grandniece|in-law|stepson|stepdaughter|brother|sister|cousin'

# url base
wiki_url_base = "https://en.wikipedia.org" # wikipedia
imdb_base_url = 'https://www.imdb.com/title/'

# wikipedia language setting
wiki_wiki = wiki.Wikipedia('en')


# ia = imdb.Cinemagoer()

In [161]:
top_250_mov_url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
top_250_tv_url = "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"
pop_mov_url = "https://www.imdb.com/search/title/?title_type=feature"
pop_tv_url = "https://www.imdb.com/search/title/?title_type=tv_series"

In [4]:
##########################
###     FUNCTIONS      ###
##########################

In [162]:
# [ input  ] words: string
# [ output ] string
# [  goal  ] replaces spaces with _
def remove_ws(words):
    no_ws = words.replace(" ", "_")
    return no_ws

In [163]:
# [ input  ] imdb url: string, type: string, class_name: string (class of list item on imdb list page)
# [ output ] dataframe
# [  goal  ] gets title, link, image, year for each title in an imdb list
def get_top_lists(url, type, class_name):

    options = Options()
    options.headless = False

    service = Service('/Users/jessicacarr/documents/j/github/are_they_a_nepo/chromedriver')
    driver = webdriver.Chrome(options=options, service=service)
    driver.get(url)

    if('search' in url):
          click_button_four_times(driver)
          print("after clicking")

    # class_name = "ipc-metadata-list-summary-item.sc-10233bc-0.iherUv.cli-parent"    
    elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, class_name))
     )
    
    print(f"Number of elements found: {len(elements)}")

    columns = ["image", "link", "title", "year", "type"]
    df = pd.DataFrame(columns=columns)

    for i, element in enumerate(elements, 1):

            try:
                img_element = element.find_element(By.CLASS_NAME, "ipc-image")
                img_src = img_element.get_attribute('src')
            except NoSuchElementException:
                img_src = ""  # Set img_src to empty string if element is not found

            a_element = element.find_element(By.CLASS_NAME, "ipc-title-link-wrapper")
            a_href = a_element.get_attribute('href')

            title_element = element.find_element(By.CLASS_NAME, "ipc-title__text")
            title = title_element.text
            title = title.split(". ",1)[1]
            print(title)

            try:
                year_element = element.find_element(By.CSS_SELECTOR, ".sc-b189961a-8.kLaxqf.cli-title-metadata-item, .sc-b189961a-8.kLaxqf.dli-title-metadata-item")
                year = year_element.text
            except NoSuchElementException:
                year = ""


            title = {
                "image" : img_src,
                "link": a_href,
                "title": title,
                "year": year,
                "type": type

            }    

            temp_df = pd.DataFrame([title])
            df = pd.concat([df, temp_df], ignore_index=True)

    driver.quit()

    return df

In [164]:
# [ input  ] driver: chromedriver
# [ output ] 
# [  goal  ] clicks on a button on an imdb page 9 times
def click_button_four_times(driver):
    click_count = 0 
    class_name = "ipc-btn.ipc-btn--single-padding.ipc-btn--center-align-content.ipc-btn--default-height.ipc-btn--core-base.ipc-btn--theme-base.ipc-btn--on-accent2.ipc-text-button.ipc-see-more__button"

    while click_count < 5:  # Loop until 9 clicks have been made
        # Wait until the button is present in the DOM and is clickable
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, class_name))
        )
        # Scroll the button into view
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        # Wait until the button is clickable
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, class_name))
        )
        # Try clicking the button
        try:
            button.click()
        except ElementClickInterceptedException:
            # If element is not clickable, try JavaScript click
            driver.execute_script("arguments[0].click();", button)
        click_count += 1  # Increment the counter after each click


In [166]:
# [ input  ] search_term: string
# [ output ] string
# [  goal  ] creates wikipedia link with the input string
def get_title_link(search_term):
    wiki_search = wikipedia.search(search_term)[0]
    wiki_search_no_ws = remove_ws(wiki_search)
    wiki_url = wiki_url_base + "/wiki/" + wiki_search_no_ws
    
    return wiki_url

In [165]:
# [ input  ] url: string
# [ output ] bs4 object
# [  goal  ] gets the soup for a url
def get_soup(url):
    data = requests.get(url, headers=headers).text
    soup = BeautifulSoup(data,'html.parser') # full page
    
    return soup

In [167]:
# [ input  ] soup: soup, parent: string, child: string
# [ output ] bs4 object
# [  goal  ] looks for element of child class in element of parent class
def find_child_in_soup(soup, parent, child):
    child = soup.find(f"{parent}",{"class":f"{child}"})
    
    return child

In [168]:
# [ input  ] soup: soup, infobox: bs4 object
# [ output ] array
# [  goal  ] determines if the relevant fields in a wikipedia infobox exist and if it does analyse infobox
def do_infobox_fields_exist(soup, infobox):
    infobox_fields = infobox.find_all('th', {'class' : 'infobox-label'})
    does_starring_field_exist = bool(re.search("Starring", str(infobox_fields)))
    does_voices_field_exist = bool(re.search("Voices of", str(infobox_fields)))
    
    if does_voices_field_exist is False and does_starring_field_exist is False:
        return [False, False]
    
    else :
        return field_results(soup, infobox, does_starring_field_exist, does_voices_field_exist)

In [169]:
# [ input  ] soup: soup, infobox: bs4, does_starring_field_exist: bool, does_voices_field_exist: bool
# [ output ] array
# [  goal  ] extracts cast list and cast links from wikipedia infobox if these exist
def field_results(soup, infobox, does_starring_field_exist, does_voices_field_exist):
    if does_voices_field_exist is True:
        th_pattern = voice_pattern
    elif does_starring_field_exist is True:
        th_pattern = starring_pattern
        
    th = soup.find('th', string=th_pattern).parent
    li_list = th.find_all('li')
    
    cast_list = []
    cast_links = []
    
    for li in li_list:
        a = li.find('a')
        if a == None :
            cast_list.append(li.text)
            cast_links.append("")
        else :
            cast_list.append(a.text)
            cast_links.append(a['href'])
            
    if len(cast_list) == 0 :
        return [False, False]
    else :
        return [cast_list, cast_links]

In [184]:
# [ input  ] title: string, year: string, tv_or_film: string
# [ output ] array
# [  goal  ] finds wikipedia link for a given title and then extracts cast list from page
def get_cast_from_wiki(title, year, tv_or_film): 

    print(title)
    
    search_term = f"{title} {year} {tv_or_film}"
    wiki_url = get_title_link(search_term)
    
    soup = get_soup(wiki_url)
    infobox = find_child_in_soup(soup, "table", "infobox vevent")
    
    if infobox is None :
        return [[],[]]
        
    else :        
        cast_list = []
        cast_list = do_infobox_fields_exist(soup, infobox)
        
        cast_names = cast_list[0]
        cast_links = cast_list[1]
        
        return cast_list

In [171]:
# [ input  ] title_id: string (imdb id)
# [ output ] array
# [  goal  ] extracts cast from imdb title page
def title_specs(title_id):
    
    url = imdb_base_url + title_id
    
    soup = get_soup(url)
    
    # CAST
    # grab top cast section of imdb page
    t = soup.find_all("div", {"class": "sc-bfec09a1-7 dpBDvu"})
    s = soup.find_all("a", {"class": "sc-bfec09a1-1 gCQkeh"})
    
    cast_list = []
    cast_links = []
    
    # get cast names
    for a in s:
        cast_list.append(str(a.string))
        cast_links.append("")
        
    return [cast_list, cast_links]

In [178]:
# [ input  ] name: string
# [ output ] bool
# [  goal  ] checks if wikipedia page exists
def does_wiki_exist(name) :
    wiki_page = wiki_wiki.page(name.title())
    does_page_exist = wiki_page.exists()
    
    return does_page_exist

In [248]:
# [ input  ] url: string
# [ output ] string (img link)
# [  goal  ] extracts image from wikipedia infobox
def get_infobox_image(url):
    soup = get_soup(url)

    image = soup.find("div", {"class": "mw-body-content"}).find("img", {"class": "mw-file-element"})
    image_str = str(image)
    
    try:
        found = re.search('src="//(.+?)"', image_str).group(1)
    except AttributeError:
        found = ''
        
    return found

In [180]:
# [ input  ] url: string
# [ output ] subj_img: string, parent_data: object
# [  goal  ] extracts data from wikipedia infobox
def extract_infobox_data(url):
    soup = get_soup(url)
    
    bio_vcard = find_child_in_soup(soup, "table", "infobox biography vcard")
    vcard = find_child_in_soup(soup, "table", "infobox vcard")
    
    if bio_vcard is None and vcard is None : 
        return '', []
    else :
        if bio_vcard is not None :
            infobox = bio_vcard
        else :
            infobox = vcard
        
        blue_links = parse_infobox_parent_fields(soup, infobox)
        subj_img = get_infobox_image(url)
        
        parent_data = []
        
        if len(blue_links) != 0:
            for link in blue_links:
                name = link.partition('/wiki/')[2]
                name_wws = name.replace("_", " ")
                
                wiki_link = wiki_url_base + "/wiki/" + name
                image = get_infobox_image(wiki_link)
                
                parent = {
                    'name': name_wws,
                    'image': image,
                    'link': wiki_link
                }
                
                parent_data.append(parent)
        
        return subj_img, parent_data
    

In [173]:
# [ input  ] soup: soup, infobox: bs4
# [ output ] array
# [  goal  ] extracts parent / relative data from a wikipedia infobox
def parse_infobox_parent_fields(soup, infobox):
    
    infobox_fields = infobox.find_all('th', {'class' : 'infobox-label'})
    
    try :
        does_parent_field_exist = bool(re.search("Parent", str(infobox_fields)))
        does_relative_field_exist = bool(re.search("Relative", str(infobox_fields)))

        if does_parent_field_exist is False and does_relative_field_exist is False:
            return [] # parent field not listed in infobox -> not a nepo baby 

        else :
            if does_parent_field_exist is True : 
                return get_parent_links(soup)
            else :
                return get_rel_links(soup)
            
    except AttributeError:
        pass


In [174]:
# [ input  ] soup: soup
# [ output ] array
# [  goal  ] extracts parent data from a wikipedia infobox
def get_parent_links(soup) : 
    parent_field = soup.find('th', string=parent_pattern).parent
    parent_a_tags = parent_field.find_all('a')
                        
    if len(parent_a_tags) == 0 :
        return [] # parents listed in infobox but not linked -> not a nepo baby

    else : # nepo baby!
        parent_wiki_list = []

        for link in parent_field.find_all('a'):
            parent_wiki = link.get('href')
            parent_wiki_link = wiki_url_base + parent_wiki
            parent_wiki_list.append(parent_wiki_link)

            parent_wiki_list[:] = [x for x in parent_wiki_list if "cite_note" not in x] # cited entries are in <a href> tags so remove those links here

    return parent_wiki_list

In [175]:
# [ input  ] soup: soup
# [ output ] array
# [  goal  ] extracts relative data from a wikipedia infobox
def get_rel_links(soup) :
    relative_field = soup.find('th', string=relative_pattern).parent
    relative_td_tags = relative_field.find_all('td')

    list_of_relatives = re.split('</li>|<br/>', str(relative_td_tags))

    parent_wiki_list = []

    if len(relative_field.find_all('a')) == 0 :
        return []
    else :
        for i in range(len(list_of_relatives)):
            if bool(re.search(not_nepo_relationships, list_of_relatives[i])) is True:
                pass

            else :
                try :
                    href_match = re.search(r'href=\"(.*)\" title=', list_of_relatives[i])
                    parent_wiki = href_match.group(1)
                    parent_wiki_link = wiki_url_base + parent_wiki
                    parent_wiki_list.append(parent_wiki_link)

                except AttributeError :
                    pass  

        if len(parent_wiki_list) == 0 :
            return []
        else :
            return parent_wiki_list


In [176]:
# [ input  ] name: string, wiki_name_link: string
# [ output ] object
# [  goal  ] determines if name is a nepo babe based on their wikipedia page
def are_they_a_nepo(name, wiki_name_link = ""):
    
    poi = {'name': name,
          'image': '',
          'link': wiki_name_link,
          'nepo': False
          }
    
    if name is False or name == 'nan' :
        return poi
    
    if wiki_name_link != "" :
        url = wiki_url_base + wiki_name_link
        does_page_exist = True
    else :
        subject = remove_ws(name.title())
        url = wiki_url_base + "/wiki/" + subject
        print(url)
        does_page_exist = does_wiki_exist(name)
        
    if does_page_exist is False :
        poi['parents'] = []

    else : 
        subj_image, parent_data = extract_infobox_data(url)
        poi['link'] = url
        poi['image'] = subj_image
        poi['parents'] = parent_data
        
        if len(poi['parents']) != 0 :
            poi['nepo'] = True

    return poi

In [279]:
# [ input  ] nepo_links: object
# [ output ] Pandas series
# [  goal  ] extracts value from dictionary to create new columsn in df
def extract_nepo_values(nepo_links):
    if isinstance(nepo_links, dict):
        return pd.Series({
            'nepo_image': nepo_links.get('image', ''),
            'nepo_link': nepo_links.get('link', ''),
            'nepo': nepo_links.get('nepo', '')
        })
    return pd.Series({'nepo_image': '', 'nepo_link': '', 'nepo': ''})


In [ ]:
##########################
###     EXECUTING      ###
##########################

In [ ]:
# retrieve titles from imdbs top and pop 250 tv and films
top_250_mov = get_top_lists(top_250_mov_url, "film", "ipc-metadata-list-summary-item.sc-10233bc-0.iherUv.cli-parent")
top_250_tv = get_top_lists(top_250_tv_url, "tv", "ipc-metadata-list-summary-item.sc-10233bc-0.iherUv.cli-parent")
pop_250_mov = get_top_lists(pop_mov_url, "film", "sc-74bf520e-3.klvfeN.dli-parent")
pop_250_tv = get_top_lists(pop_tv_url, "tv", "sc-74bf520e-3.klvfeN.dli-parent")

In [186]:
# join all dataframes and extract title id from the title url 
dfs_to_concat = [top_250_mov, top_250_tv, pop_250_mov, pop_250_tv]
df = pd.concat(dfs_to_concat)
df['id'] = df['link'].str.extract(r'/title/(tt\d+)/\?ref')
df = df.drop_duplicates(subset=['id'])

,image,link,title,year,type,id
0,https://m.media-amazon.com/images/M/MV5BNDE3OD...,https://www.imdb.com/title/tt0111161/?ref_=cht...,The Shawshank Redemption,1994,film,tt0111161
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,https://www.imdb.com/title/tt0068646/?ref_=cht...,The Godfather,1972,film,tt0068646
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,https://www.imdb.com/title/tt0468569/?ref_=cht...,The Dark Knight,2008,film,tt0468569
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,https://www.imdb.com/title/tt0071562/?ref_=cht...,The Godfather Part II,1974,film,tt0071562
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,https://www.imdb.com/title/tt0050083/?ref_=cht...,12 Angry Men,1957,film,tt0050083
...,...,...,...,...,...,...
242,https://m.media-amazon.com/images/M/MV5BMmRkMW...,https://www.imdb.com/title/tt13111078/?ref_=sr...,Lioness,2023–,tv,tt13111078
243,https://m.media-amazon.com/images/M/MV5BMjQ4ZD...,https://www.imdb.com/title/tt10691770/?ref_=sr...,Helluva Boss,2019–,tv,tt10691770
244,https://m.media-amazon.com/images/M/MV5BMTg0Nz...,https://www.imdb.com/title/tt0071007/?ref_=sr_...,Little House on the Prairie,1974–1983,tv,tt0071007
246,https://m.media-amazon.com/images/M/MV5BOTZkNj...,https://www.imdb.com/title/tt2934286/?ref_=sr_...,Halo,2022–,tv,tt2934286


In [187]:
# retrieve cast list and cast links for every title from wikipedia
df.loc[:,'cast'] = df.apply(lambda row : get_cast_from_wiki(row['title'], row['year'], row['type']), axis=1)
df[['cast_names','cast_links']] = pd.DataFrame(df.cast.tolist(), index= df.index)
df = df.drop('cast', axis=1)

The Shawshank Redemption
The Godfather
The Dark Knight
The Godfather Part II
12 Angry Men
Schindler's List
The Lord of the Rings: The Return of the King
Pulp Fiction
The Lord of the Rings: The Fellowship of the Ring
The Good, the Bad and the Ugly
Forrest Gump
The Lord of the Rings: The Two Towers
Fight Club
Inception
Star Wars: Episode V - The Empire Strikes Back
The Matrix
Goodfellas
One Flew Over the Cuckoo's Nest
Se7en
Interstellar
It's a Wonderful Life
Seven Samurai
The Silence of the Lambs
Saving Private Ryan
City of God
Life Is Beautiful
The Green Mile
Terminator 2: Judgment Day
Star Wars: Episode IV - A New Hope
Back to the Future
Spirited Away
The Pianist
Dune: Part Two
Parasite
Psycho
Gladiator
Spider-Man: Across the Spider-Verse
The Lion King
The Departed
Léon: The Professional
American History X
Whiplash
The Prestige
Grave of the Fireflies
Harakiri
The Usual Suspects
Casablanca
The Intouchables
Cinema Paradiso
Modern Times
Rear Window
Once Upon a Time in the West
Alien
City 

In [194]:
# if wiki didn't list the cast then get it from imdb
df['cast_len'] = df['cast_names'].str.len()
wiki_df = df[(df['cast_len'] != 0) & (df['cast_len'].notnull())]
imdb_df = df[(df['cast_len'] == 0) | (df['cast_len'].isnull())]

imdb_df.loc[:, "cast"] = imdb_df.apply(lambda row : title_specs(row['id']), axis=1) # get top cast from imdb
imdb_df[['cast_names','cast_links']] = pd.DataFrame(imdb_df.cast.tolist(), index= imdb_df.index)
imdb_df = imdb_df.drop('cast', axis=1)

/var/folders/fb/pmmnfq35613dv6n8fynzxt8w0000gn/T/ipykernel_91199/773092319.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_df.loc[:, "cast"] = imdb_df.apply(lambda row : title_specs(row['id']), axis=1) # get top cast from imdb
/var/folders/fb/pmmnfq35613dv6n8fynzxt8w0000gn/T/ipykernel_91199/773092319.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_df[['cast_names','cast_links']] = pd.DataFrame(imdb_df.cast.tolist(), index= imdb_df.index)


In [250]:
# merge the wiki and imdb dfs back together
df = pd.concat([wiki_df, imdb_df]).reset_index(drop=True)

# redo cast length to capture the imdb cast lists
df['cast_len'] = df['cast_names'].str.len()
df = df.explode(['cast_names', 'cast_links']).reset_index(drop=True)
df = df.dropna(subset=['cast_names'])

In [252]:
# retrieve wikipedia blue link data on each cast member
df.loc[:,"nepo_links"] = df.apply(lambda row : are_they_a_nepo(row['cast_names'], row['cast_links']), axis=1)

https://en.wikipedia.org/wiki/Henry_Fonda
https://en.wikipedia.org/wiki/Jun_Tatara
https://en.wikipedia.org/wiki/Daniel_Zettel
https://en.wikipedia.org/wiki/Roberto_Benigni
https://en.wikipedia.org/wiki/Takashi_Naito
https://en.wikipedia.org/wiki/Takehiko_Ono
https://en.wikipedia.org/wiki/Niketa_Calame
https://en.wikipedia.org/wiki/Charlie_Chaplin
https://en.wikipedia.org/wiki/Charlie_Chaplin
https://en.wikipedia.org/wiki/Mel_Gibson
https://en.wikipedia.org/wiki/Matt_Damon
https://en.wikipedia.org/wiki/Ben_Affleck
https://en.wikipedia.org/wiki/Yordanos_Shiferaw
https://en.wikipedia.org/wiki/Boluwatife_Bankole
https://en.wikipedia.org/wiki/Kawthar_Al_Haddad
https://en.wikipedia.org/wiki/Fadi_Kamel_Youssef
https://en.wikipedia.org/wiki/Nour_El_Husseini
https://en.wikipedia.org/wiki/Alaa_Chouchnieh
https://en.wikipedia.org/wiki/Cedra_Izam
https://en.wikipedia.org/wiki/Nadine_Labaki
https://en.wikipedia.org/wiki/Joseph_Jimbazian
https://en.wikipedia.org/wiki/Farah_Hasno
https://en.wikipedi

In [281]:
df[['cast_image', 'cast_link', 'cast_nepo']] = df['nepo_links'].apply(extract_nepo_values)
df = df[["id", "title", "year", "link", "image", "type", "cast_len", "cast_names", "cast_links", "cast_image", "cast_link", "cast_nepo"]]

,image,link,title,year,type,id,cast_names,cast_links,cast_len,nepo_links,nepo_image,nepo_link,nepo,cast_image,cast_link,cast_nepo
0,https://m.media-amazon.com/images/M/MV5BNDE3OD...,https://www.imdb.com/title/tt0111161/?ref_=cht...,The Shawshank Redemption,1994,film,tt0111161,Tim Robbins,/wiki/Tim_Robbins,7,"{'name': 'Tim Robbins', 'image': 'upload.wikim...",upload.wikimedia.org/wikipedia/commons/thumb/6...,https://en.wikipedia.org/wiki/Tim_Robbins,False,upload.wikimedia.org/wikipedia/commons/thumb/6...,https://en.wikipedia.org/wiki/Tim_Robbins,False
1,https://m.media-amazon.com/images/M/MV5BNDE3OD...,https://www.imdb.com/title/tt0111161/?ref_=cht...,The Shawshank Redemption,1994,film,tt0111161,Morgan Freeman,/wiki/Morgan_Freeman,7,"{'name': 'Morgan Freeman', 'image': 'upload.wi...",upload.wikimedia.org/wikipedia/commons/thumb/4...,https://en.wikipedia.org/wiki/Morgan_Freeman,False,upload.wikimedia.org/wikipedia/commons/thumb/4...,https://en.wikipedia.org/wiki/Morgan_Freeman,False
2,https://m.media-amazon.com/images/M/MV5BNDE3OD...,https://www.imdb.com/title/tt0111161/?ref_=cht...,The Shawshank Redemption,1994,film,tt0111161,Bob Gunton,/wiki/Bob_Gunton,7,"{'name': 'Bob Gunton', 'image': 'upload.wikime...",upload.wikimedia.org/wikipedia/commons/thumb/7...,https://en.wikipedia.org/wiki/Bob_Gunton,False,upload.wikimedia.org/wikipedia/commons/thumb/7...,https://en.wikipedia.org/wiki/Bob_Gunton,False
3,https://m.media-amazon.com/images/M/MV5BNDE3OD...,https://www.imdb.com/title/tt0111161/?ref_=cht...,The Shawshank Redemption,1994,film,tt0111161,William Sadler,/wiki/William_Sadler_(actor),7,"{'name': 'William Sadler', 'image': 'upload.wi...",upload.wikimedia.org/wikipedia/commons/thumb/a...,https://en.wikipedia.org/wiki/William_Sadler_(...,False,upload.wikimedia.org/wikipedia/commons/thumb/a...,https://en.wikipedia.org/wiki/William_Sadler_(...,False
4,https://m.media-amazon.com/images/M/MV5BNDE3OD...,https://www.imdb.com/title/tt0111161/?ref_=cht...,The Shawshank Redemption,1994,film,tt0111161,Clancy Brown,/wiki/Clancy_Brown,7,"{'name': 'Clancy Brown', 'image': 'upload.wiki...",upload.wikimedia.org/wikipedia/commons/thumb/3...,https://en.wikipedia.org/wiki/Clancy_Brown,True,upload.wikimedia.org/wikipedia/commons/thumb/3...,https://en.wikipedia.org/wiki/Clancy_Brown,True


In [283]:
df.to_csv("nepo_cast.csv")

In [254]:
##########################
###        JSON        ###
##########################

In [277]:

# Group by 'id' and aggregate the required fields
result = []
grouped = df.groupby('id')

for id, group in grouped:
    first_row = group.iloc[0]
    entry = {
        'title': first_row['title'],
        'id': id,
        'year': first_row['year'],
        'image': first_row['image'],
        'link': first_row['link'],
        'type': first_row['type'],
        'cast_len': int(first_row['cast_len']),
        'cast': group['nepo_links'].tolist()
    }
    result.append(entry)

# Convert to JSON
json_result = json.dumps(result, indent=2)


In [278]:
with open('output.json', 'w') as file:
    json.dump(result, file, indent=2)